https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sqlalchemy
sqlalchemy.__version__ 

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

engine

from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

Base

'1.3.13'

Engine(sqlite:///:memory:)

sqlalchemy.ext.declarative.api.Base

In [4]:
from sqlalchemy import Column, Integer, String
class User(Base):
     __tablename__ = 'users'

     id = Column(Integer, primary_key=True)
     name = Column(String)
     fullname = Column(String)
     nickname = Column(String)

     def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
                             self.name, self.fullname, self.nickname)

In [5]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [6]:
Base.metadata.create_all(engine)

2020-03-10 13:05:07,205 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-10 13:05:07,206 INFO sqlalchemy.engine.base.Engine ()
2020-03-10 13:05:07,209 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-10 13:05:07,210 INFO sqlalchemy.engine.base.Engine ()
2020-03-10 13:05:07,212 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-03-10 13:05:07,213 INFO sqlalchemy.engine.base.Engine ()
2020-03-10 13:05:07,214 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-03-10 13:05:07,215 INFO sqlalchemy.engine.base.Engine ()
2020-03-10 13:05:07,218 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2020-03-10 13:05:07,219 INFO sqlalchemy.engine.base.Engine ()
2020-03-10 13:05:07,220 INFO sqlalchemy.engine.base.Engine COMMIT


In [72]:
# from sqlalchemy import Sequence
# Column(Integer, Sequence('user_id_seq'), primary_key=True)

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [8]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine, autoflush=False)

In [9]:
session = Session()
session.add(ed_user)

In [10]:
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [11]:
session.commit()

2020-03-10 13:06:06,884 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-10 13:06:06,888 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-03-10 13:06:06,889 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2020-03-10 13:06:06,892 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
session.query(User).filter_by(name='ed').first()

2020-03-10 13:06:11,271 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-10 13:06:11,273 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-03-10 13:06:11,274 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [13]:
our_user = session.query(User).filter_by(name='ed').first() 
our_user

2020-03-10 13:06:35,094 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-03-10 13:06:35,096 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [14]:
our_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [15]:
ed_user is our_user
ed_user
our_user

True

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [16]:
session.add_all([
     User(name='wendy', fullname='Wendy Williams', nickname='windy'),
     User(name='mary', fullname='Mary Contrary', nickname='mary'),
     User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [17]:
ed_user.nickname = 'eddie'

In [18]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [19]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [20]:
session.commit()

2020-03-10 13:07:03,446 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2020-03-10 13:07:03,447 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2020-03-10 13:07:03,449 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-03-10 13:07:03,450 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-03-10 13:07:03,451 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-03-10 13:07:03,452 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-03-10 13:07:03,453 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-03-10 13:07:03,454 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-03-10 13:07:03,456 INFO sqlalchemy.engine.base.Engine COMMIT
